# Creating Custom Domain Plugins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Excelsior2026/reasoning-core/blob/main/examples/04_custom_domain.ipynb)

This notebook shows you how to create your own domain plugins for specialized reasoning extraction.

## What You'll Learn

- Understanding the BaseDomain interface
- Creating a custom domain (Legal example)
- Defining domain-specific terminology
- Implementing reasoning patterns
- Testing your custom domain
- Best practices for domain design

In [ ]:
from reasoning_core import ReasoningAPI
from reasoning_core.plugins.base_domain import BaseDomain
from reasoning_core.extractors.concept_extractor import Concept
from reasoning_core.extractors.relationship_mapper import Relationship
from typing import List, Dict, Any
import re

## Understanding BaseDomain

Every domain plugin extends `BaseDomain` and implements these methods:

```python
class BaseDomain:
    def get_name(self) -> str
    def get_terminology_mapping(self) -> Dict[str, List[str]]
    def get_reasoning_patterns(self) -> Dict[str, Any]
    def extract_concepts(self, text: str) -> List[Concept]
    def identify_relationships(self, concepts: List[Concept], text: str) -> List[Relationship]
    def generate_questions(self, content: Dict[str, Any]) -> List[str]
```

## Example: Legal Domain Plugin

Let's create a domain for legal reasoning:

In [ ]:
class LegalDomain(BaseDomain):
    """Domain plugin for legal reasoning extraction."""
    
    def get_name(self) -> str:
        return "legal"
    
    def get_terminology_mapping(self) -> Dict[str, List[str]]:
        """Legal terminology by category."""
        return {
            'statutes': [
                'statute', 'law', 'code', 'regulation', 'act',
                'USC', 'CFR', 'ordinance', 'rule'
            ],
            'cases': [
                'precedent', 'case law', 'holding', 'decision',
                'ruling', 'judgment', 'opinion', 'v.', 'vs.'
            ],
            'legal_concepts': [
                'jurisdiction', 'standing', 'liability', 'damages',
                'breach', 'duty', 'negligence', 'intent', 'mens rea',
                'actus reus', 'burden of proof', 'standard of review'
            ],
            'parties': [
                'plaintiff', 'defendant', 'appellant', 'appellee',
                'petitioner', 'respondent', 'counsel', 'court'
            ],
            'outcomes': [
                'affirmed', 'reversed', 'remanded', 'dismissed',
                'granted', 'denied', 'sustained', 'overruled'
            ]
        }
    
    def get_reasoning_patterns(self) -> Dict[str, Any]:
        """Legal reasoning patterns."""
        return {
            'case_analysis': {
                'pattern': ['parties', 'facts', 'legal_concepts', 'statutes', 'holding', 'outcomes'],
                'description': 'Case law analysis workflow'
            },
            'statutory_interpretation': {
                'pattern': ['statutes', 'legal_concepts', 'cases', 'outcomes'],
                'description': 'Statute application to facts'
            },
            'precedent_application': {
                'pattern': ['cases', 'legal_concepts', 'facts', 'outcomes'],
                'description': 'Applying precedent to new situation'
            }
        }
    
    def extract_concepts(self, text: str) -> List[Concept]:
        """Extract legal concepts from text."""
        concepts = []
        terminology = self.get_terminology_mapping()
        
        for concept_type, terms in terminology.items():
            for term in terms:
                # Case-insensitive search
                pattern = rf'\b{re.escape(term)}\b'
                matches = re.finditer(pattern, text, re.IGNORECASE)
                
                for match in matches:
                    start = match.start()
                    # Extract context (50 chars before and after)
                    context_start = max(0, start - 50)
                    context_end = min(len(text), match.end() + 50)
                    context = text[context_start:context_end].strip()
                    
                    concepts.append(Concept(
                        text=match.group(),
                        type=concept_type,
                        confidence=0.85,
                        context=context,
                        position=start
                    ))
        
        return concepts
    
    def identify_relationships(self, concepts: List[Concept], text: str) -> List[Relationship]:
        """Identify legal relationships."""
        relationships = []
        
        # Define legal relationship patterns
        rel_patterns = [
            ('cases', 'legal_concepts', 'establishes'),
            ('statutes', 'legal_concepts', 'defines'),
            ('parties', 'outcomes', 'receives'),
            ('legal_concepts', 'outcomes', 'results_in'),
        ]
        
        for source_type, target_type, rel_type in rel_patterns:
            sources = [c for c in concepts if c.type == source_type]
            targets = [c for c in concepts if c.type == target_type]
            
            for source in sources:
                for target in targets:
                    # Check proximity
                    distance = abs(source.position - target.position)
                    if distance < 200:
                        relationships.append(Relationship(
                            source=source,
                            target=target,
                            type=rel_type,
                            confidence=0.7,
                            evidence=f"{source.text} {rel_type} {target.text}"
                        ))
        
        return relationships
    
    def generate_questions(self, content: Dict[str, Any]) -> List[str]:
        """Generate legal analysis questions."""
        questions = []
        
        if 'cases' in content:
            for case in content['cases'][:3]:
                questions.append(f"What is the holding in {case}?")
                questions.append(f"How does {case} apply to these facts?")
        
        if 'statutes' in content:
            for statute in content['statutes'][:2]:
                questions.append(f"What are the elements of {statute}?")
        
        if 'legal_concepts' in content:
            for concept in content['legal_concepts'][:2]:
                questions.append(f"How is {concept} defined in this jurisdiction?")
        
        return questions

print('✅ LegalDomain class created!')

## Test the Custom Domain

In [ ]:
# Create API with legal domain
legal_api = ReasoningAPI(domain=LegalDomain())

# Test with legal text
legal_text = """
In Smith v. Jones, the Court held that the defendant breached their duty of care,
resulting in negligence liability. The statute 42 USC 1983 defines civil rights violations.
The plaintiff established standing based on direct injury. The Court affirmed the lower
court's ruling and awarded damages to the plaintiff.
"""

result = legal_api.process_text(legal_text)

print(f"Concepts: {len(result['concepts'])}")
print(f"Relationships: {len(result['relationships'])}")
print(f"Questions: {len(result['questions'])}\n")

print("=== Extracted Legal Concepts ===\n")
for concept in result['concepts']:
    print(f"  {concept['text']:20} | {concept['type']}")

## View Generated Questions

In [ ]:
print("=== Legal Analysis Questions ===\n")
for i, q in enumerate(result['questions'], 1):
    print(f"{i}. {q}")

## Best Practices for Custom Domains

### 1. Terminology

- **Be comprehensive**: Include variations, abbreviations, synonyms
- **Organize logically**: Group related terms by category
- **Domain expert input**: Validate with subject matter experts

### 2. Reasoning Patterns

- **Reflect real workflows**: Model actual expert thinking
- **Multiple patterns**: Support different reasoning types
- **Clear descriptions**: Document what each pattern represents

### 3. Concept Extraction

- **Context awareness**: Include surrounding text
- **Confidence scoring**: Reflect extraction certainty
- **Deduplication**: Handle repeated terms

### 4. Relationship Identification

- **Domain-specific types**: Use meaningful relationship names
- **Proximity heuristics**: Nearby concepts often relate
- **Pattern matching**: Look for linguistic cues

### 5. Question Generation

- **Pedagogically sound**: Ask meaningful questions
- **Varied difficulty**: Mix easy and complex questions
- **Context-aware**: Reference specific extracted content

### 6. Testing

- **Diverse texts**: Test on various document types
- **Edge cases**: Handle unusual inputs gracefully
- **Performance**: Optimize for speed and accuracy

## Ideas for Custom Domains

### Education
- **Nursing**: Patient care protocols and clinical workflows
- **Engineering**: Design patterns and technical specifications
- **Finance**: Investment analysis and risk assessment

### Professional Services
- **Consulting**: Problem-solution-value frameworks
- **Project Management**: Agile workflows and methodologies
- **Customer Success**: Onboarding and retention patterns

### Research
- **Scientific**: Hypothesis-experiment-conclusion chains
- **Historical**: Events-causes-effects analysis
- **Literary**: Theme-symbol-interpretation patterns

## Contributing Your Domain

Want to contribute your domain to reasoning-core?

1. **Create** your domain plugin
2. **Add tests** in `tests/test_{domain}_domain.py`
3. **Document** with examples
4. **Submit** a pull request

See [CONTRIBUTING.md](https://github.com/Excelsior2026/reasoning-core/blob/main/CONTRIBUTING.md) for details.

## Next Steps

- **[05_knowledge_graphs.ipynb](05_knowledge_graphs.ipynb)** - Advanced graph operations
- **[Contributing Guide](https://github.com/Excelsior2026/reasoning-core/blob/main/CONTRIBUTING.md)** - Submit your domain
- **[API Documentation](https://github.com/Excelsior2026/reasoning-core)** - Full API reference